In [102]:
import re


def process_input(text):
    rules, parts = text.split("\n\n")
    rules_keys = [re.findall(r"(.+)\{", rule)[0] for rule in rules.split("\n")]
    rules_values = [
        re.findall(r"\{(.+)\}", rule)[0].split(",") for rule in rules.split("\n")
    ]
    rules_dict = dict(zip(rules_keys, rules_values))
    parts = [part[1:-1].split(",") for part in parts.split("\n")]
    part_dict = [dict([(cat[0], int(cat[2:])) for cat in part]) for part in parts]
    return rules_dict, part_dict


def check_rule(part, rule):
    if rule[1] == ">":
        part_val = part[rule[0]]
        rule_val = int(re.findall(r"\>(.+)\:", rule)[0])
        return part_val > rule_val
    else:
        rule_val = int(re.findall(r"\<(.+)\:", rule)[0])
        part_val = part[rule[0]]
        return part_val < rule_val


def check_workflow(part, workflow_name):
    rules = rules_dict[workflow_name]
    for rule in rules:
        if len(rule) == 1:
            return rule
        elif rule[1] in ["<", ">"]:
            if check_rule(part, rule):
                return re.findall(r"\:(.+)$", rule)[0]
        else:
            return rule
    return part, rules


def check_part(part, start_workflow="in"):
    current_workflow = start_workflow
    while current_workflow not in ["R", "A"]:
        current_workflow = check_workflow(part, current_workflow)
    return current_workflow


def sum_values(parts):
    to_sum = []
    for part in parts:
        if check_part(part) == "A":
            to_sum.append(sum(part.values()))
    return sum(to_sum)


def split_boundary(boundary, rule):
    new_u_boundary = boundary.copy()
    new_l_boundary = boundary.copy()
    if rule[1] == ">":
        if int(re.findall(r"\>(.+)\:", rule)[0]) + 1 > new_l_boundary[rule[0] + "l"]:
            new_l_boundary[rule[0] + "l"] = int(re.findall(r"\>(.+)\:", rule)[0]) + 1
        if int(re.findall(r"\>(.+)\:", rule)[0]) < new_u_boundary[rule[0] + "u"]:
            new_u_boundary[rule[0] + "u"] = int(re.findall(r"\>(.+)\:", rule)[0])
        return new_u_boundary, False, new_l_boundary, True
    else:
        if int(re.findall(r"\<(.+)\:", rule)[0]) > new_l_boundary[rule[0] + "l"]:
            new_l_boundary[rule[0] + "l"] = int(re.findall(r"\<(.+)\:", rule)[0])
        if int(re.findall(r"\<(.+)\:", rule)[0]) - 1 < new_u_boundary[rule[0] + "u"]:
            new_u_boundary[rule[0] + "u"] = int(re.findall(r"\<(.+)\:", rule)[0]) - 1
        return new_u_boundary, True, new_l_boundary, False


def boundary_step(boundary):
    if boundary["workflow"] in ["A", "R"]:
        return [boundary]
    rule = rules_dict[boundary["workflow"]][boundary["count"]]
    if len(rule) == 1:
        boundary["workflow"] = rule
        return [boundary]
    elif rule[1] in ["<", ">"]:
        b1, bool1, b2, bool2 = split_boundary(boundary, rule)
        if bool1:
            b1["workflow"] = re.findall(r"\:(.+)$", rule)[0]
            b1["count"] = 0
            b2["count"] += 1
        if bool2:
            b2["workflow"] = re.findall(r"\:(.+)$", rule)[0]
            b2["count"] = 0
            b1["count"] += 1
        return [b1, b2]
    else:
        boundary["workflow"] = rule
        boundary["count"] = 0
        return [boundary]


def boundary_workflow(boundary):
    next_step = boundary_step(boundary)
    if len(next_step) == 1:
        if next_step[0]["workflow"] in ["R", "A"]:
            return next_step
        else:
            return boundary_workflow(next_step[0])
    return boundary_workflow(next_step[0]) + boundary_workflow(next_step[1])


def process_boundaries(initial_boundary):
    splits = [
        split
        for split in boundary_workflow(initial_boundary)
        if split["workflow"] == "A"
    ]
    vals = [
        (
            (split["xu"] - split["xl"] + 1)
            * (split["mu"] - split["ml"] + 1)
            * (split["au"] - split["al"] + 1)
            * (split["su"] - split["sl"] + 1)
        )
        for split in splits
    ]
    return sum(vals)


rules_dict, part_dict = process_input(open("inputs/day19.txt", "r").read())
initial_boundary = {
    "workflow": "in",
    "count": 0,
    "xu": 4000,
    "xl": 1,
    "mu": 4000,
    "ml": 1,
    "au": 4000,
    "al": 1,
    "su": 4000,
    "sl": 1,
}

print(f"Part 1 Answer: {sum_values(part_dict)}")
print(f"Part 2 Answer: {process_boundaries(initial_boundary)}")

Part 1 Answer: 425811
Part 2 Answer: 131796824371749
